In [ ]:
def query_check(query):
    client = boto3.client('athena', region_name='us-east-1')
    response = client.start_query_execution(
                QueryString=query,
                QueryExecutionContext={'Database': "temp_tables"},
                ResultConfiguration={'OutputLocation': f"s3://bucket/aws-athena-ci"},
                WorkGroup='CI'
        )
    query_status_state = 'RUNNING'
    while query_status_state == 'QUEUED' or query_status_state == 'RUNNING':
        query_status = client.get_query_execution(QueryExecutionId=response["QueryExecutionId"])['QueryExecution']["Status"]
        query_status_state = query_status["State"]
        if query_status_state == 'FAILED' or query_status_state == 'CANCELLED':
            raise Exception(f'Failed query {query} with the following execption {query_status["StateChangeReason"]}')
        time.sleep(1)


In [ ]:
def test_queries_synatx():
    with engine.connect() as con:
        queries_list = glob.glob('**/*.sql',recursive=True)
        for query_file in queries_list:
            with open(query_file) as f:
                query = f.read()
                test_query = f"Explain {query}"
                qc = query_check(test_query)


In [ ]:
name: CI tests for MWAA
on:
  push:
    branches:
      - master
  pull_request:
    branches:
      - master
jobs:
  unit-testing:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout repo content
      uses: actions/checkout@v2 # checkout the repository content to github runner
      - name: Set up Python 3.8
      uses: actions/setup-python@v2
      with:
            python-version: 3.8
      - name: Install Dependencies
        run: |
          python -m pip install -r deploy/ci_requirements.txt
      - name: Test with pytest
        run: |
          pytest .